## <u> Convert some spectrum to an audio spectrum and generate the sound</u>

First, we import relevant modules:

In [ ]:
%matplotlib inline
%reload_ext autoreload 
%autoreload 2
import matplotlib.pyplot as plt
import ffmpeg as ff
import wavio as wav
from strauss.sonification import Sonification
from strauss.sources import Events
from strauss import channels
from strauss.score import Score
import numpy as np
from strauss.generator import Sampler, Synthesizer
import IPython.display as ipd
from IPython.core.display import display
import os
from scipy.signal import savgol_filter

### Discrete wavelengths: A first example with emission line galaxies

As a first example, we choose some discrete wavelengths to sonify, representing emission lines from the galaxy **NGC1569** (or optionally **NGC4670**, commented out). [Example spectrum can be seen here](http://astronomy.nmsu.edu/nicole/teaching/ASTR505/lectures/lecture26/pics/Irr.gif). Here the frequencies are in nanometers (nm). The strenth of these lines above the *continuum* is mapped to the volume of these frequencies

In [ ]:

wlens = [372, 386, 410, 433, 485, 495, 
         500, 587, 630, 640, 654, 655, 
         658, 671]

#NGC1569
vols  = [3.21, 1.38, 1.09, 1.86, 5.54, 9.2,
         26.77, 1.51, 1.30, 1.13, 3.00, 21.72, 
         1.91, 2.64]

#NGC4670
#vols  = [21.07, 1.67, 1.54, 3.7, 9.22, 7.31,
#         21.8, 0.61, 0.65, 0.00, 8.97, 21.84, 2.35,
#         3.17]

The frequencies volume of each line is mapped from `'0'`-`'100'` where strings indicate the percentile range. For volum numerical `0` implies zero amplitude, and `'100'` is the 100th percentile, i.e. the volume of the strongest line (here the oxygen OIII line, at 500nm)  

In [ ]:
maplims =  {'pitch' : ('0', '100'),
            'volume' : (-0.5, '100')}

We then convert the light ***wavelengths*** to sound ***frequencies***. While we want all these frequences to be in the audible range (~20-20,000 Hz) this mapping is somewhat arbitrary. What's the best range to use for this example?

In [ ]:
# the maximum and minimum sound frequency we use to map our tones... what happens if we change this?
FMAX = 3000
FMIN = 100

freqs = (1/np.array(wlens))
# Start at 5 Hz
freqs *= (FMAX-FMIN)/(freqs.max()-freqs.min())
freqs += FMIN-freqs.min()

# make sure pitches are ordered from lowest to highest frequency
chords=[freqs[::-1]]
print(chords)

We are now ready to set up the sonification! 

We load the special preset `'spectraliser'` which will generate the tones we want. We decide on a length and system for the simulation (make it short for quicker generation, using `mono` only generates a single channel so also speeds things up as we don't need stereo information for this example)

In [ ]:
generator = Synthesizer()
generator.load_preset('spectraliser')
length = "0m 4s"
system='mono'
score =  Score(chords, length)

# higher pitch -> higher *frequency* (not wavelength)
ps = np.array(freqs)
vs = np.array(vols)

events = Events(['pitch', 'volume'])
events.raw_mapping = dict(zip(['pitch', 'volume'],
                           [ps, vs]))

events.apply_mapping_functions(map_lims=maplims)
events.n_sources = ps.shape[0]
soni = Sonification(score, events, generator, system)

We render the sonification...

In [ ]:
soni.render()

Then display and listen to it, also plotting the spectral lines showing the 'partials' that constitute the sound

In [ ]:
soni.notebook_display()
plt.vlines(ps, vs*0, vs)
plt.ylim(0,30)
plt.semilogx()
plt.xlabel('Pitch Frequency (Hz)')
plt.ylabel('Amplitude')

and save the WAV file if you like (as e.g. `some/path/file.wav`)

In [ ]:
soni.save(FILEPATH_AS_A_STRING.WAV)

### Continuous wavelengths: A full stellar spectrum

We can extend this approach to render a continuous spectrum using a grid of wavelengths. First we can load the example M-class stellar spectrum (or O-class by changing the filename)

In [ ]:
data = np.genfromtxt('../data/datasets/M_spectrum.txt')
volumes = data[:,1]
wlens   = data[:,0]

# A different maximum and minimum sound frequency this time... what happens if we change this?
FMAX = 10000
FMIN = 50
freqs = (1/np.array(wlens))
freqs *= (FMAX-FMIN)/(freqs.max()-freqs.min())
freqs += FMIN-freqs.min()

# make sure pitches are ordered from lowest to highest frequency
chords=[freqs[::-1]]
print(chords)

plt.plot(freqs, volumes)
plt.semilogx()
plt.xlabel('Pitch Frequency (Hz)')
plt.ylabel('Amplitude')

In [ ]:
# smooth the spectrum, this will give something like the continuum
volumes_smooth = savgol_filter(volumes, 51, 3)

# subtracting away the 'continuum' 
volumes_negfeats = -np.clip((volumes - volumes_smooth), -np.inf, 0)#/volumes_smooth_O
volumes_posfeats = np.clip((volumes - volumes_smooth), 0, np.inf)#/volumes_smooth_O

plt.plot(freqs, volumes, c='k', alpha=0.3)
plt.plot(freqs, volumes_smooth - 0.6*volumes_O.max())
plt.plot(freqs, volumes_negfeats - 1.2*volumes_O.max())
plt.plot(freqs, volumes_posfeats - 1.8*volumes_O.max())
plt.semilogx()
plt.xlabel('Pitch Frequency (Hz)')
plt.ylabel('Amplitude')

Same mapping limits as the first example

In [ ]:
maplims =  {'pitch' : ('0', '100'),
            'volume' : (0, '100')}

Setup sonification generator

In [ ]:
generator = Synthesizer()
generator.load_preset('spectraliser')
length = "0m 4s"
system='mono'
score =  Score(chords, length)

As an aside, one feature of the spectraliser preset is that the phases of the different frequencies are randomised, using the special keyword `'random'`, this can also just be set to e.g. a constant value so that all the pitches have the same phases. This value is from 0-1 and represents the fraction of a sine-wave's cycle that the tones start on. I wonder why this is set to random, and what happens if we change the phase to a constant value (e.g. `0`)?

In [ ]:
print("Phase setting before: ", generator.preset['oscillators']['osc1']['phase'])
generator.modify_preset({'oscillators':{'osc1':{'phase':'random'}}})
print("Phase setting after: ", generator.preset['oscillators']['osc1']['phase'])

Now setup the sonification again, choosing one of the 'volumes' to use...

In [ ]:
# higher pitch -> higher *frequency* (not wavelength)
ps = np.array(freqs)

# listen to the negative features (absorption) by default, can change this
vs = np.array(volumes_negfeats)

events = Events(['pitch', 'volume'])
events.raw_mapping = dict(zip(['pitch', 'volume'],
                           [ps, vs]))

events.apply_mapping_functions(map_lims=maplims)
events.n_sources = ps.shape[0]
soni2 = Sonification(score, events, generator, system)

...and render (this will take longer as we are using many more frequencies, remember the shorter the sonification the quicker the render)

In [ ]:
soni2.render()

In [ ]:
soni2.notebook_display()
plt.plot(ps,vs)
plt.semilogx()
plt.xlabel('Pitch Frequency (Hz)')
plt.ylabel('Amplitude')

and save the WAV file if you like (as e.g. `some/path/file.wav`)

In [ ]:
soni2.save(FILEPATH_AS_A_STRING.WAV)

As a bonus, what if we generated some simple functions to sonify as spectra?

In [ ]:
vol_flat = np.ones(freqs.shape)
vol_single = (freqs == FMIN).astype(float)
vol_shallow = (freqs/5000)**-0.5
vol_steep = (freqs/5000)**-2

In [ ]:
# higher pitch -> higher *frequency* (not wavelength)
ps = np.array(freqs)

# listen to vol_steep by default
vs = np.array(vol_steep)

events = Events(['pitch', 'volume'])
events.raw_mapping = dict(zip(['pitch', 'volume'],
                           [ps, vs]))

events.apply_mapping_functions(map_lims=maplims)
events.n_sources = ps.shape[0]
soni3 = Sonification(score, events, generator, system)

In [ ]:
soni3.render()

In [ ]:
soni3.notebook_display()
plt.plot(ps,vs)
plt.semilogx()
plt.xlabel('Pitch Frequency (Hz)')
plt.ylabel('Amplitude')

and save the WAV file if you like (as e.g. `some/path/file.wav`)

In [ ]:
soni3.save(FILEPATH_AS_A_STRING.WAV)